Tests for GPU access

In [1]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13475581536767056811
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 8998066342
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2353459627848847161
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:0a:00.0, compute capability: 7.5"
]


Creates a dataset made of random images and key presses

In [48]:
import numpy as np

def save_data(data, labels):
    file_name = 'Data\\training_data_test.npz'
    np.savez_compressed(file_name,data=data,labels=labels)

In [49]:
data = []

for n in range(5):
    print("Generating image " + str(n) + "...")
    data.append([])
    for i in range(5):
        data[n].append([])
        for j in range(270):
            data[n][i].append([])
            for k in range(480):
                data[n][i][j].append([])
                for l in range(3):
                    data[n][i][j][k].append(np.uint8(np.random.randint(255)))
                    
data = np.array(data)

print(data.shape)

Generating image 0...
Generating image 1...
Generating image 2...
Generating image 3...
Generating image 4...
(5, 5, 270, 480, 3)


In [50]:
labels = []

for n in range(5):
    print("Creating label " + str(n) + "...")
    i = np.random.choice(list(range(9)))
    
    if i ==0:
        labels.append([0,0,0,0])
    if i ==1:
        labels.append([1,0,0,0])
    if i ==2:
        labels.append([0,1,0,0])
    if i ==3:
        labels.append([0,0,1,0])
    if i ==4:
        labels.append([0,0,0,1])
    if i ==5:
        labels.append([1,0,1,0])
    if i ==6:
        labels.append([1,0,0,1])
    if i ==7:
        labels.append([0,1,1,0])
    if i ==9:
        labels.append([0,1,0,1])

    
labels = np.array(labels)
print(labels.shape)

Creating label 0...
Creating label 1...
Creating label 2...
Creating label 3...
Creating label 4...
(3, 4)


In [51]:
save_data(data, labels)

Continuously presses w while controlling the game

In [10]:
# For controlling the game
from inputsHandler import select_key
from getkeys import key_check
import time
import tensorflow as tf

In [11]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

In [ ]:
start = 0

while True:
    if start == 0:
        if 'K' in key_check():
            start = 1
        continue
            
    if not 'J' in key_check():
        if time.time()%50 < 25:
            select_key(3)
        else:
            select_key(4)
    else:
        break
    time.sleep(5)

Runs a version of the model that works on random images in the game

In [71]:
import glob
from inputsHandler import select_key
from getkeys import key_check
import time
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2 
from IPython.display import clear_output
from sys import stdout

In [72]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

In [73]:
def key_press(key):
    if key == 1:
        return'A'
    if key == 2:
        return'D'
    if key == 3:
        return'W'
    if key == 4:
        return'S'
    if key == 5:
        return'AW'
    if key == 6:
        return'AS'
    if key == 7:
        return'DW'
    if key == 8:
        return'DS'
    return 'none'

In [74]:
def mse(imageA, imageB):

    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err

In [75]:
# Loads the model from the file
obj_model = load_model('yolo.h5', compile=False)

In [76]:
#By Lavanya Shukla on Kaggle.

#Custom class for functionality needed to extract bounding boxes from the YOLO model

class BoundBox:
    def __init__(self, xmin, ymin, xmax, ymax, objness = None, classes = None):
        self.xmin = xmin
        self.ymin = ymin
        self.xmax = xmax
        self.ymax = ymax
        self.objness = objness
        self.classes = classes
        self.label = -1
        self.score = -1

    def get_label(self):
        if self.label == -1:
            self.label = np.argmax(self.classes)

        return self.label

    def get_score(self):
        if self.score == -1:
            self.score = self.classes[self.get_label()]
 
        return self.score
 
def _sigmoid(x):
    return 1. / (1. + np.exp(-x))
 
def decode_netout(netout, anchors, obj_thresh, net_h, net_w):
    grid_h, grid_w = netout.shape[:2]
    nb_box = 3
    netout = netout.reshape((grid_h, grid_w, nb_box, -1))
    nb_class = netout.shape[-1] - 5
    boxes = []
    netout[..., :2]  = _sigmoid(netout[..., :2])
    netout[..., 4:]  = _sigmoid(netout[..., 4:])
    netout[..., 5:]  = netout[..., 4][..., np.newaxis] * netout[..., 5:]
    netout[..., 5:] *= netout[..., 5:] > obj_thresh

    for i in range(grid_h*grid_w):
        row = i / grid_w
        col = i % grid_w
        for b in range(nb_box):
            # 4th element is objectness score
            objectness = netout[int(row)][int(col)][b][4]
            if(objectness.all() <= obj_thresh): continue
            # first 4 elements are x, y, w, and h
            x, y, w, h = netout[int(row)][int(col)][b][:4]
            x = (col + x) / grid_w # center position, unit: image width
            y = (row + y) / grid_h # center position, unit: image height
            w = anchors[2 * b + 0] * np.exp(w) / net_w # unit: image width
            h = anchors[2 * b + 1] * np.exp(h) / net_h # unit: image height
            # last elements are class probabilities
            classes = netout[int(row)][col][b][5:]
            box = BoundBox(x-w/2, y-h/2, x+w/2, y+h/2, objectness, classes)
            boxes.append(box)
    return boxes

def correct_yolo_boxes(boxes, image_h, image_w, net_h, net_w):
    new_w, new_h = net_w, net_h
    for i in range(len(boxes)):
        x_offset, x_scale = (image_w - new_w)/2./net_w, float(new_w)/net_w
        y_offset, y_scale = (image_h - new_h)/2./net_h, float(new_h)/net_h
        boxes[i].xmin = int((boxes[i].xmin - x_offset) / x_scale * image_w)
        boxes[i].xmax = int((boxes[i].xmax - x_offset) / x_scale * image_w)
        boxes[i].ymin = int((boxes[i].ymin - y_offset) / y_scale * image_h)
        boxes[i].ymax = int((boxes[i].ymax - y_offset) / y_scale * image_h)

def _interval_overlap(interval_a, interval_b):
    x1, x2 = interval_a
    x3, x4 = interval_b
    if x3 < x1:
        if x4 < x1:
            return 0
        else:
            return min(x2,x4) - x1
    else:
        if x2 < x3:
            return 0
        else:
            return min(x2,x4) - x3

def bbox_iou(box1, box2):
    intersect_w = _interval_overlap([box1.xmin, box1.xmax], [box2.xmin, box2.xmax])
    intersect_h = _interval_overlap([box1.ymin, box1.ymax], [box2.ymin, box2.ymax])
    intersect = intersect_w * intersect_h
    w1, h1 = box1.xmax-box1.xmin, box1.ymax-box1.ymin
    w2, h2 = box2.xmax-box2.xmin, box2.ymax-box2.ymin
    union = w1*h1 + w2*h2 - intersect
    return float(intersect) / union

def do_nms(boxes, nms_thresh):
    if len(boxes) > 0:
        nb_class = len(boxes[0].classes)
    else:
        return
    for c in range(nb_class):
        sorted_indices = np.argsort([-box.classes[c] for box in boxes])
        for i in range(len(sorted_indices)):
            index_i = sorted_indices[i]
            if boxes[index_i].classes[c] == 0: continue
            for j in range(i+1, len(sorted_indices)):
                index_j = sorted_indices[j]
                if bbox_iou(boxes[index_i], boxes[index_j]) >= nms_thresh:
                    boxes[index_j].classes[c] = 0

# load and prepare an image
def load_image_pixels(filename, shape):
    # load the image to get its shape
    image = load_img(filename)
    width, height = image.size
    # load the image with the required size
    image = load_img(filename, target_size=shape)
    # convert to numpy array
    image = img_to_array(image)
    # scale pixel values to [0, 1]
    image = image.astype('float32')
    image /= 255.0
    # add a dimension so that we have one sample
    image = expand_dims(image, 0)
    return image, width, height

# get all of the results above a threshold
def get_boxes(boxes, labels, thresh):
    v_boxes, v_labels, v_scores = list(), list(), list()
    # enumerate all boxes
    for box in boxes:
        # enumerate all possible labels
        for i in range(len(labels)):
            # check if the threshold for this label is high enough
            if box.classes[i] > thresh:
                v_boxes.append(box)
                v_labels.append(labels[i])
                v_scores.append(box.classes[i]*100)
                # don't break, many labels may trigger for one box
    return v_boxes, v_labels, v_scores

# draw all results
def draw_boxes(filename, v_boxes, v_labels, v_scores):
    # load the image
    data = pyplot.imread(filename)
    # plot the image
    pyplot.imshow(data)
    # get the context for drawing boxes
    ax = pyplot.gca()
    # plot each box
    for i in range(len(v_boxes)):
        box = v_boxes[i]
        # get coordinates
        y1, x1, y2, x2 = box.ymin, box.xmin, box.ymax, box.xmax
        # calculate width and height of the box
        width, height = x2 - x1, y2 - y1
        # create the shape
        rect = Rectangle((x1, y1), width, height, fill=False, color='white')
        # draw the box
        ax.add_patch(rect)
        # draw text and score in top left corner
        label = "%s (%.3f)" % (v_labels[i], v_scores[i])
        pyplot.text(x1, y1, label, color='white')
    # show the plot
    pyplot.show()

In [77]:
def get_sub_images(IMAGE):
    #declare labels for the model
    labels = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck","boat"]
    
    #get image size and model paramaters
    WIDTH, HEIGHT = 416,416
    anchors = [[116,90, 156,198, 373,326], [30,61, 62,45, 59,119], [10,13, 16,30, 33,23]]
    class_threshold = 0.3
    
    #load original image
    width, height = (IMAGE.shape[0], IMAGE.shape[1])
    
    # load the image with the required size
    image = cv2.resize(IMAGE, dsize=(WIDTH, HEIGHT), interpolation=cv2.INTER_CUBIC)
    
    #normalize image and add a dimension
    image = image.astype('float32')
    image /= 255.0
    image = np.expand_dims(image, 0)
    
    yhat = obj_model.predict(image)
    
    # Create boxes
    boxes = list()
    for i in range(len(yhat)):
        # decode the output of the network
        boxes += decode_netout(yhat[i][0], anchors[i], class_threshold, HEIGHT, WIDTH)
    
    # correct the sizes of the bounding boxes for the shape of the image
    correct_yolo_boxes(boxes, height, width, HEIGHT, WIDTH)
    
    # suppress non-maximal boxes
    do_nms(boxes, 0.5)
    
    # get the details of the detected objects
    v_boxes, v_labels, v_scores = get_boxes(boxes, labels, class_threshold)
    
    objects = np.zeros_like(IMAGE)
    background = IMAGE[:]
    
    for x in range(len(v_boxes)):
        box = v_boxes[x]
        # get coordinates
        y1, x1, y2, x2 = box.ymin, box.xmin, box.ymax, box.xmax
        # Loop over area of each box
        for i in range(x1,x2):
            for j in range(y1,y2):
                # Data is channels last, so we copy and erase whole channels here
                # objects contains all values within the box
                objects[j][i] = IMAGE[j][i][:]
                # background gets this coordinate erased and is left with the remainder of the image
                background[j][i] = np.zeros_like(IMAGE[j][i])
    
    return (objects, background)

In [78]:
def reshape_custom_X(data, verbose = 1):
    reshaped = np.zeros((data.shape[0], 5, 270, 480, 3), dtype=np.float32)
    for i in range(0, data.shape[0]):
        for j in range(0, 5):
            if (verbose == 1):
                clear_output(wait=True)
                stdout.write('Reshaped image: ' + str(i))
                stdout.flush()
            reshaped[i][j] = data[i][j]/255.
            
    return reshaped

def reshape_custom_y(data):
    reshaped = np.zeros((data.shape[0], 9), dtype=np.float32)
    for i in range(0, data.shape[0]):
        if np.array_equal(data[i] , [0,0,0,0]):
            reshaped[i][0] = 1.
        elif np.array_equal(data[i] , [1,0,0,0]):
            reshaped[i][1] = 1.
        elif np.array_equal(data[i] , [0,1,0,0]):
            reshaped[i][2] = 1.
        elif np.array_equal(data[i] , [0,0,1,0]):
            reshaped[i][3] = 1.
        elif np.array_equal(data[i] , [0,0,0,1]):
            reshaped[i][4] = 1.
        elif np.array_equal(data[i] , [1,0,1,0]):
            reshaped[i][5] = 1.
        elif np.array_equal(data[i] , [1,0,0,1]):
            reshaped[i][6] = 1.
        elif np.array_equal(data[i] , [0,1,1,0]):
            reshaped[i][7] = 1.
        elif np.array_equal(data[i] , [0,1,0,1]):
            reshaped[i][8] = 1.
    return reshaped

In [81]:
enable_evasion = False

model = load_model('Model\\'+'nonsense')

while True:
    if 'K' in key_check():
        break


files = glob.glob("Data\\*.npz")


actual_file = 0
for fil in files:
    #time.sleep(10)
    actual_file = actual_file + 1
    with np.load(fil) as data:
        training_data_images = data['data']
        training_data_labels = data['labels']

    #print("\n Balancing data...")
    #number = number_instances_per_class(training_data)
    #training_data = np.array(balance_data(number))
    
    train_x = training_data_images[:]
    train_y = training_data_labels[:]

    del training_data_images, training_data_labels
    
    train_objects = []
    train_background = []
    for i in range(len(train_x)):
        train_objects.append([])
        train_background.append([])
        for j in range(5):
            temp1,temp2 = get_sub_images(train_x[i][j])
            train_objects[i].append(temp1)
            train_background[i].append(temp2)
    
    X_train = [np.array(train_objects), np.array(train_background)]
    y_train = reshape_custom_y(np.array(train_y))
    del train_x, train_y, train_objects, train_background
    
    while True:
        #time.sleep(10)
        last_time = time.time()
        for i in range(len(X_train[0])):
            p = np.argmax(model.predict([np.expand_dims(X_train[0][i], axis=0), np.expand_dims(X_train[1][i], axis=0)]))

            if not 'J' in key_check():
                 select_key(p)

            if enable_evasion:
                score = mse(X_train[0][0],X_train[0][4])
                if score < 1000:
                    select_key(4)
                    time.sleep(1)
                    if np.random.rand()>0.5:
                        select_key(6)
                    else:
                        select_key(8)
                    time.sleep(0.2)

            time_act = time.time()
            clear_output(wait=True)
            stdout.write('Actions per second: ' + str(1/(time_act-last_time)) + '\n')
            if enable_evasion:
                stdout.write('Diference from img 1 to img 5: ' + str(score))
            stdout.flush()
            last_time = time.time()

            if cv2.waitKey(1) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break

Actions per second: 0.7276103427674112


KeyboardInterrupt: 

In [ ]:
print(X_train[0][0].shape)
p = np.argmax(model.predict([np.expand_dims(X_train[0][0], axis=0), np.expand_dims(X_train[1][1], axis=0)]))
print(p)

In [ ]:
k